In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from scipy import stats
agw2017 = pd.read_stata("agw2017")
agw2013 = pd.read_stata("agw2013")
agw2007 = pd.read_stata("agw2007")

gez2017 = pd.read_stata("gez2017")
gez2013 = pd.read_stata("gez2013")
gez2007 = pd.read_stata("gez2007")




In [40]:
print(agw2013["b26ogb"])

0       225000.000000
1       220000.000000
2                 NaN
3            0.000000
4                 NaN
5       255000.000000
6       150000.000000
7                 NaN
8       175000.000000
9       200000.000000
10      260000.015625
11      200000.000000
12                NaN
13           0.000000
14           0.000000
15                NaN
16           0.000000
17      400000.000000
18                NaN
19      545000.000000
20           0.000000
21           0.000000
22      245000.000000
23           0.000000
24           0.000000
25      125000.000000
26      175000.000000
27      200000.000000
28      479999.968750
29      400000.000000
            ...      
2011              NaN
2012              NaN
2013         0.000000
2014         0.000000
2015    260000.015625
2016              NaN
2017              NaN
2018    255000.000000
2019              NaN
2020         0.000000
2021              NaN
2022         0.000000
2023    370000.000000
2024              NaN
2025      

Vragenlijst Wonen en Hypotheken
B26Og eigenaar van woning (1=ja, 0=nee)
B26Hy hypotheken op de woning
B26Vz cash value levensverzekering hypotheek woning
B27Og eigenaar van tweede woning (1=ja, 0=nee)
B27Hy hypotheken op tweede woning
B27Vz cash value levensverzekering hypotheek tweede woning

### Eerst maak ik de Net-worth variabele

Dit doe ik dmv een functie die alle bezittingen (b1b, b2b, ..., b30b) bij elkaar optelt en daar de schulden (s1b, s2b, ..., s8b, x1b) vanaf trekt.

In [41]:
def calcWealth(row):
    wealth = (row["b1b"] + row["b2b"] + row["b3b"] + row["b4b"] + row["b6b"] + row["b7b"] + row["b8b"] + row["b11b"]
              + row["b12b"] + row["b13b"] + row["b14b"] + row["b15b"] + row["b16b"] + row["b17b"] + row["b18b"] + 
             row["b19ogb"] + row["b19hyb"] + row["b19vzb"] + row["b20b"] + row["b21b"] + row["b22b"] + row["b23b"] +
             row["b24b"] + row["b25b"] + row["b28b"] + row["b29b"] + row["b30b"])
    # Door te checken of b26ogb > -1 zorg ik dat enkel rows zonder NaN hier worden gebruikt. Ze zijn of volledig Nan of niet
    if row["b26ogb"] > -1:
        wealth += row["b26ogb"] + row["b26vzb"] + row["b27ogb"] + row["b27vzb"]
    return wealth

In [42]:
def calcDebt(row):
    debt = (row["s1b"] + row["s2b"] + row["s3b"] + row["s4b"] + row["s5b"] + row["s6b"] + row["s7b"] + row["s8b"] +
           row["x1b"]) 
    # Door te checken of b26ogb > -1 zorg ik dat enkel rows zonder NaN hier worden gebruikt. Ze zijn of volledig Nan of niet
    if row["b26ogb"] > -1:
        debt += row["b26hyb"] + row["b27hyb"]
    
    return debt

In [43]:
def calcNetWorth(row):
    netWorth = row["wealth"] - row["debt"]
    return netWorth

In [44]:
agw2017["debt"] = agw2017.apply(calcDebt, axis = 1)
agw2013["debt"] = agw2013.apply(calcDebt, axis = 1)
agw2007["debt"] = agw2007.apply(calcDebt, axis = 1)




In [45]:
agw2017["wealth"] = agw2017.apply(calcWealth, axis = 1)
agw2013["wealth"] = agw2013.apply(calcWealth, axis = 1)
agw2007["wealth"] = agw2007.apply(calcWealth, axis = 1)



In [46]:
agw2017["netWorth"] = agw2017.apply(calcNetWorth, axis = 1)
agw2013["netWorth"] = agw2013.apply(calcNetWorth, axis = 1)
agw2007["netWorth"] = agw2007.apply(calcNetWorth, axis = 1)

In [47]:
# Counter om aantal positieve en negatieve networths te tellen
pos = 0
neg = 0
zero = 0

# Arrays om de negatieven en positieven networths bij te houden
negatives = []
positives = []

# Bepaald voor iedere row of de networth positief of negatief is en zet deze waarde in een array
for x in range(0, agw2013.shape[0] - 1):
    if agw2013.loc[x,"netWorth"] > 0:
        positives.append(agw2013.loc[x, "netWorth"])
        pos += 1
    elif agw2013.loc[x, "netWorth"] < 0:
        negatives.append(agw2013.loc[x, "netWorth"])
        neg += 1
    else:
        zero += 1
        
print("aantal positieve networth: " + str(pos) +  " aantal negatieve networth: " + str(neg))
print("aantal met net worth nul (waarschijnlijk lege entries): " + str(zero))

aantal positieve networth: 1722 aantal negatieve networth: 219
aantal met net worth nul (waarschijnlijk lege entries): 99


In [48]:
# negatives.sort()
# positives.sort()

# plt.hist(positives)
# plt.title("positive net worths")
# plt.show()

# plt.hist(positives, range = (0, 250000))
# plt.title("positive net worths met max wealth 250k")
# plt.show()

# plt.hist(negatives)
# plt.title("negative net worths")
# plt.show()

# plt.hist(negatives, range = (-25000, 0))
# plt.title("negative net worths with -25k max debt")
# plt.show()

# Hier boven:

het feit dat bezit tot 10x zo veel gaat als schuld, kan leiden tot ene verschil in effect op  gezondheid (bijvoorbeeld 10x minder sterk oid) Misschien hier nog iets mee doen!

In [49]:
merged2017 = pd.merge(gez2017, agw2017, on = "personid")
merged2013 = pd.merge(gez2013, agw2013, on = "personid")
merged2007 = pd.merge(gez2007, agw2007, on = "personid")


In [50]:
merged = pd.merge(merged2017, merged2013, on = "personid")
merged = pd.merge(merged, merged2007, on = "personid")


In [51]:
def selfAssedHealth(row):
    healthValues = {"Excellent" : 1, "Good" : 1, "Fair" : 1, "Not so good" : 0, "Poor" : 0,
                    "excellent" : 1, "good" : 1, "fair" : 1, "not so good" : 0, "poor" : 0} 
    
    return healthValues[row["gez3"]]


http://www.statsmodels.org/stable/discretemod.html

In [52]:
merged2017["health"] = merged2017.apply(selfAssedHealth, axis = 1)
merged2013["health"] = merged2013.apply(selfAssedHealth, axis = 1)
merged2007["health"] = merged2007.apply(selfAssedHealth, axis = 1)

In [53]:
def dummyGroup(row):
    if row["b26oga"] == 1:
        return 1
    else:
        return 0
        

In [54]:
merged2017["treated"] = merged2017.apply(dummyGroup, axis = 1)
merged2013["treated"] = merged2013.apply(dummyGroup, axis = 1)
merged2007["treated"] = merged2007.apply(dummyGroup, axis = 1)


In [55]:
merged2017["time"] = 1
merged2013["time"] = 1
merged2007["time"] = 0

In [56]:
def interaction(row):
    return (row["time"] * row["treated"])

In [57]:
merged2017["DID"] = merged2017.apply(interaction, axis = 1)
merged2013["DID"] = merged2013.apply(interaction, axis = 1)
merged2007["DID"] = merged2007.apply(interaction, axis = 1)

In [58]:
merged2017["2013"] = 0
merged2017["2017"] = 1
merged2013["2013"] = 1
merged2013["2017"] = 0
merged2007["2013"] = 0
merged2007["2017"] = 0

In [59]:
frames0713 = [merged2007, merged2013]
frames0717 = [merged2007, merged2017]
frames071317 = [merged2007, merged2013, merged2017]
list0713 = pd.concat(frames0713)
list0717 = pd.concat(frames0717)
list071317 = pd.concat(frames071317)


In [60]:
merged2013["time"] = 0
merged2013["DID"] = merged2013.apply(interaction, axis = 1)
frames1317 = [merged2013, merged2017]
list1317 = pd.concat(frames1317)

In [61]:
list0713.to_stata("list0713.dta")
list0717.to_stata("list0717.dta")
list071317.to_stata("list071317.dta")
list1317.to_stata("list1317.dta")

/home/niels/anaconda3/lib/python3.6/site-packages/pandas/io/stata.py:2086: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    b'2013'   ->   _2013
    b'2017'   ->   _2017

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  warnings.warn(ws, InvalidColumnName)


In [62]:
def nHealthy(data):
    healthy = 0
    unhealthy = 0
    for x in range(0, data.shape[0] - 1):
        if data.loc[x, "health"] == 1:
            healthy += 1
        else:
            unhealthy += 1
    print("aantal healthy: " + str(healthy) + " en aantal unhealthy: " + str(unhealthy))

In [63]:
nHealthy(merged2013)

KeyError: 'the label [selfAssessedHealth] is not in the [index]'

In [64]:
nHealthy(merged2007)

KeyError: 'the label [selfAssessedHealth] is not in the [index]'

In [65]:
nHealthy(merged2017)

KeyError: 'the label [selfAssessedHealth] is not in the [index]'

In [66]:
frames = [merged2007, merged2013, merged2017]
langeLijst = pd.concat(frames)

In [67]:
langeLijst.to_stata("langeLijst.dta")

/home/niels/anaconda3/lib/python3.6/site-packages/pandas/io/stata.py:2086: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    b'2013'   ->   _2013
    b'2017'   ->   _2017

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  warnings.warn(ws, InvalidColumnName)


In [68]:
result = sm.ols(formula="health ~ time + treated + DID", data=list1317).fit()


In [69]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 health   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     7.821
Date:                Wed, 09 Jan 2019   Prob (F-statistic):           3.34e-05
Time:                        14:25:06   Log-Likelihood:                 380.15
No. Observations:                3864   AIC:                            -752.3
Df Residuals:                    3860   BIC:                            -727.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9513      0.008    125.810      0.0

In [70]:
result = sm.ols(formula="health ~ time + treated + DID", data=list0717).fit()


In [71]:
list1317["wealth"]

0       412746.859375
1       237406.250000
2          800.000000
3        50318.999512
4            0.000000
5       358478.898438
6       162000.000000
7        40960.553467
8       288229.515625
9       242859.997803
10      569046.000000
11      214337.000000
12        7340.000000
13        6313.500000
14        1500.000000
15       14641.999512
16         768.125000
17      416060.999756
18        4041.999756
19      574500.000000
20       38525.000000
21        9158.000000
22      249125.000000
23        1300.000000
24        4700.000000
25      227466.279297
26      246304.000000
27      500485.968262
28      431530.000000
29      598796.329346
            ...      
2185    424500.000000
2186     20474.000000
2187    318170.031006
2188    239897.463501
2189    260000.015625
2190     15000.000000
2191     35851.000000
2192         0.000000
2193    378354.750000
2194    143957.867188
2195     13170.661621
2196      6000.000000
2197    392384.888550
2198      6250.000000
2199    20

In [18]:
merged2017.to_stata("merged_data2017.dta")
merged2013.to_stata("merged_data2013.dta")
merged2007.to_stata("merged_data2007.dta")

In [ ]:
merged.to_stata("merged_data.dta")

In [ ]:
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
